In [ ]:
! pip install langchain-community pypdf

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bijd4kj45kEod3cat2HExjBls4Q4w', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f81872ae-a0e7-4df4-889e-6f1996c65967-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
from langchain_community.document_loaders import PyPDFLoader

# file_path = "../langchain-projects/mml-book.pdf"
file_path = "../langchain-projects/GitLab Repository Management.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

88


In [5]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)



{'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2013-11-21T10:31:14+05:30', 'moddate': '2013-11-21T10:44:41+05:30', 'trapped': '/False', 'source': '../langchain-projects/GitLab Repository Management.pdf', 'total_pages': 88, 'page': 0, 'page_label': 'Cover'}


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=882, chunk_overlap=20, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

183

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [8]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[-0.032804619520902634, 0.014470345340669155, -0.009233088232576847, -0.0017981246346607804, -0.015952294692397118, 0.009993460960686207, -0.007669670507311821, 0.024657778441905975, -0.041742872446775436, -0.0027854444924741983]


In [ ]:
! pip install -qU langchain-pinecone

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

api_key = os.environ.get("PINECONE_API_KEY")
if not api_key:
    raise ValueError("Please set the PINECONE_API_KEY environment variable.")

pc = Pinecone(api_key=api_key)

index = pc.Index(
    index_name="rag-index",
    host="https://rag-index-4h6dcep.svc.aped-4627-b74a.pinecone.io",
)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [10]:
ids = vector_store.add_documents(documents=all_splits)

In [11]:
results = vector_store.similarity_search(
    "Who is the author of the book?"
)

print(results)

[]


In [12]:
max_len = max(len(doc.page_content.encode('utf-8')) for doc in all_splits)
print(f"Largest chunk size in bytes: {max_len}")

Largest chunk size in bytes: 893


In [ ]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("How to manage the Roles and the permissions in GitLab?")
from langchain.chains import RetrievalQA

# Wrap your vector store as a retriever
retriever = vector_store.as_retriever()

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
)

# Now you can ask questions using the chain
result = qa_chain.invoke({"query": "How to manage the Roles and the permissions in GitLab?"})
print("Answer:", result["result"])
print("Source documents:", result["source_documents"])
# Print the top result's score and document
if results:
    doc, score = results[0]
    print(f"Score: {score}\n")
    print(doc)
else:
    print("No results found.")

In [17]:
# ...existing code...
from langchain.chains import RetrievalQA

# Wrap your vector store as a retriever
retriever = vector_store.as_retriever()

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
)

# Now you can ask questions using the chain
result = qa_chain.invoke({"query": "How to manage the Roles and the permissions in GitLab?"})
print("Answer:", result["result"])
print("Source documents:", result["source_documents"])
# ...existing code...

Answer: To manage roles and permissions in GitLab, you can follow these steps:

1. Understand the available roles: GitLab offers roles like Guest, Reporter, Developer, Maintainer, and Owner, each with different levels of access and permissions.

2. Add users: You can add users to your GitLab project, and they will automatically gain access to the repositories assigned to their team.

3. Configure permissions: Assign appropriate roles to users based on the level of access they require. You can control functions like pushing to repositories and access to specific branches.

4. Manage teams and groups: Combine multiple users into teams to streamline permissions and collaboration.

5. For managing SSH keys: You can revoke a user's keys by having them delete it from their list of SSH keys in their profile. If the user can't do it themselves, you can manually delete the key from the authorized_keys file (/home/git/.ssh/authorized_keys) on the server-side.

By following these steps, you can e